In [4]:
import carla #the sim library itself

# connect to the sim 
client = carla.Client('localhost', 2000)
#world = client.get_world()
world = client.load_world('Town02')
map=world.get_map()
#print(world)

In [31]:
#global planner without u turn

import time
import numpy as np


map=world.get_map()
def key_exists(key, dict_of_dicts):
    for i in dict_of_dicts:
        if key in list(i.keys()):
            return True
    return False

waypoint_tuple_list = map.get_topology()
id_map = dict()
id_map_v = dict()
road_id_to_edge = dict()
graph={}
for waypoint in waypoint_tuple_list:
    x1, y1, x2, y2 = np.round([waypoint[0].transform.location.x, waypoint[0].transform.location.y,waypoint[1].transform.location.x, waypoint[1].transform.location.y], 0)
    road_id, section_id, lane_id = waypoint[0].road_id, waypoint[0].section_id, waypoint[0].lane_id
    if (x1,y1) not in id_map:
        id_map[(x1,y1)] = len(id_map)
        id_map_v[len(id_map_v)]=(x1,y1)
    if (x2,y2) not in id_map:
        id_map[(x2,y2)] = len(id_map)
        id_map_v[len(id_map_v)]=(x2,y2)
    if id_map[(x1,y1)] not in graph :
        graph[id_map[(x1,y1)]]={id_map[(x2,y2)]:waypoint[0]}
    else:
        if id_map[(x2,y2)] not in graph[id_map[(x1,y1)]]:
            graph[id_map[(x1,y1)]].update({id_map[(x2,y2)]:waypoint[0]})
    n1 = id_map[(x1,y1)]
    n2 = id_map[(x2,y2)]
    if road_id not in road_id_to_edge:
        road_id_to_edge[road_id] = dict()
    if section_id not in road_id_to_edge[road_id]:
        road_id_to_edge[road_id][section_id] = dict()
    road_id_to_edge[road_id][section_id][lane_id] = (n1, n2)

def dist(s,e):
    return abs(s[0]-e[0])+abs(s[1]-e[1])
def get_path(start,end):
    wa_start=map.get_waypoint(start)
    road_start_id, section_start_id, lane_start_id = wa_start.road_id, wa_start.section_id, wa_start.lane_id
    (n1_start, n2_start)=road_id_to_edge[road_start_id][section_start_id][lane_start_id] 
    
    wa_end=map.get_waypoint(end)
    road_end_id, section_end_id, lane_end_id = wa_end.road_id, wa_end.section_id, wa_end.lane_id
    (n1_end, n2_end)=road_id_to_edge[road_end_id][section_end_id][lane_end_id]

    rout=[]
    if((n1_end, n2_end)!=(n1_start, n2_start)) or dist([wa_start.transform.location.x,wa_start.transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y])<dist([wa_start.next(1)[0].transform.location.x,wa_start.next(1)[0].transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y]):
        (x1,y1),(x2,y2) = id_map_v[n2_start],id_map_v[n1_end]
        g = len(wa_start.next_until_lane_end(1))
        f=g+dist([x1,y1],[x2,y2])
        open = [[f,g,dist([x1,y1],[x2,y2]), x1, y1]]
        found = False 
        f_distance={node:float('inf') for node in graph}
        f_distance[n2_start]=f
    
        came_from={node:None for node in graph}
        came_from[n2_start]=n2_start
        #came_from={'s':None}
        #came_from={'su':None}
        #came_from={'n':None}
        #came_from={'nu':None}
        #came_from[n2_start]='s'
        
        while not found:
            if len(open) == 0:
                return "Fail"
                break
            else:
                min_ele = open[0][0]
                index=0
                for i in range(1, len(open)):
                    if open[i][0] < min_ele:
                        min_ele = open[i][0]
                        index=i
                next=open.pop(index)
                x = next[3]
                y = next[4]
                g = next[1]
                for i in graph[id_map[(x,y)]]:
                    (xf,yf)=id_map_v[i]
                    g2=len(graph[id_map[(x,y)]][i].next_until_lane_end(1))+g
                    f2=g2+dist([xf,yf],[x2,y2])
                    if came_from[i]==None:
                        open.append([f2,g2,dist([xf,yf],[x2,y2]), xf, yf])
                        f_distance[id_map[(xf,yf)]]=f2
                        came_from[id_map[(xf,yf)]]=id_map[(x,y)]
                        
                        if xf == x2 and yf == y2:
                            found = True
                            #came_from['n']=id_map[(x2,y2)]
                            #return came_from
                            wa=map.get_waypoint(carla.Location(x=x2,y=y2,z=0))
                            rout.append(wa)
                            while dist([wa.transform.location.x,wa.transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y])>1:
                                wa=wa.next(1)[0]
                                rout.append(wa)
                            while n1_end != n2_start:
                                n=came_from[n1_end]
                                ws=graph[n][n1_end].next_until_lane_end(1)
                                ws.extend(rout)
                                rout=ws.copy()
                                n1_end=n
                            ws=map.get_waypoint(carla.Location(x=wa_start.transform.location.x,y=wa_start.transform.location.y,z=0)).next_until_lane_end(1)
                            ws.extend(rout)
                            rout=ws.copy()
                            break
    else:
        rout.append(wa_start)
        while dist([wa_start.transform.location.x,wa_start.transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y])>1:
            wa_start=wa_start.next(1)[0]
            rout.append(wa_start)
        rout.append(wa_end)
    return rout


In [65]:
import sys
sys.path.append('/home/youmad55/Downloads/CARLA_0.9.15/PythonAPI/carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner
grp = GlobalRoutePlanner(world.get_map(), 1)
ws1=grp.trace_route(start,end)
for w in ws1:
    world.debug.draw_string(w[0].transform.location, '*', draw_shadow=False,
                color=carla.Color(r=0, g=255, b=0), life_time=60.0,
                persistent_lines=True)

In [19]:
for i in range(len(id_map)):
    wa=carla.Location(x=list(id_map.keys())[i][0],y=list(id_map.keys())[i][1],z=0)
    world.debug.draw_string(wa, '%d'%i, draw_shadow=False,
            color=carla.Color(r=0, g=0, b=255), life_time=600.0,
            persistent_lines=True)

In [21]:
import sys
sys.path.append('/home/youmad55/Downloads/CARLA_0.9.15/PythonAPI/carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner
grp = GlobalRoutePlanner(world.get_map(), 1)
def dist(s,e):
    return abs(s[0]-e[0])+abs(s[1]-e[1])
def astar():
    graph=[[i for i in range(12)]for j in range(12)]
    points=[[-8,140],[100,103],[193,125],[193,200],[165,235],[80,239],[135,210],[40,210],[40,290],[20,190],[95,305],[2,300]]
    start=[193,230]
    for i in range(12):
        for j in range(12):
            if i != j:
                #if (i==4 and j==3)or(i==6 and j==3)or(i==7 and j==0)or(i==8 and j==0)or(i==9 and j==0)or(i==11 and j==0):
                #    continue
                print(f"from {i} to {j}:")
                print(len(get_path(carla.Location(x=points[i][0],y=points[i][1],z=0), carla.Location(x=points[j][0],y=points[j][1],z=0))))
    f=0
    h=0
    g=0
    open=[[f,g,h,[start]]]
    path=[]
    while True:
        min_ele = open[0][0]
        index=0
        for i in range(1, len(open)):
            if open[i][0] < min_ele:
                min_ele = open[i][0]
                index=i
        next = open[index]
        open.pop(index)
        path=next[3].copy()
        for i in range(len(points)):
            if points[i] not in path:
                gp=len(get_path(carla.Location(x=path[len(path)-1][0],y=path[len(path)-1][1],z=0), carla.Location(x=points[i][0],y=points[i][1],z=0)))
                if (gp<450 and path[len(path)-1]==[95,305]) or(gp<300 and path[len(path)-1]!=[95,305]):
                    g=gp+next[1]
                    h=0
                    fpath=path.copy()
                    fpath.append(points[i])
                    for j in range(len(points)):
                        if points[j]not in fpath:
                            hj=dist(points[j],points[i])
                            #hj=len(get_path(carla.Location(x=points[i][0],y=points[i][1],z=0), carla.Location(x=points[j][0],y=points[j][1],z=0)))

                            if hj>h:
                                h=hj
                    f=2.1*h+g
                    open.append([f,g,h,fpath])
                    if len(fpath)==len(points)+1:
                        return fpath
astar()

[[193, 230],
 [193, 200],
 [193, 125],
 [100, 103],
 [-8, 140],
 [20, 190],
 [40, 210],
 [80, 239],
 [135, 210],
 [165, 235],
 [40, 290],
 [95, 305],
 [2, 300]]

In [13]:
ws1=get_path(carla.Location(x=193,y=230,z=0),carla.Location(x=193,y=200,z=0))
ws2=get_path(carla.Location(x=193,y=200,z=0),carla.Location(x=193,y=125,z=0))
ws3=get_path(carla.Location(x=193,y=125,z=0),carla.Location(x=100,y=103,z=0))
ws4=get_path(carla.Location(x=100,y=103,z=0),carla.Location(x=-8,y=140,z=0))
ws5=get_path(carla.Location(x=-8,y=140,z=0),carla.Location(x=20,y=190,z=0))
ws6=get_path(carla.Location(x=20,y=190,z=0),carla.Location(x=40,y=210,z=0))
ws7=get_path(carla.Location(x=40,y=210,z=0),carla.Location(x=80,y=239,z=0))
ws8=get_path(carla.Location(x=80,y=239,z=0),carla.Location(x=135,y=210,z=0))
ws9=get_path(carla.Location(x=135,y=210,z=0),carla.Location(x=40,y=290,z=0))
ws11=get_path(carla.Location(x=40,y=290,z=0),carla.Location(x=95,y=305,z=0))
ws12=get_path(carla.Location(x=95,y=305,z=0),carla.Location(x=165,y=235,z=0))
ws10=get_path(carla.Location(x=165,y=235,z=0),carla.Location(x=2,y=300,z=0))
wss=[ws1,ws2,ws3,ws4,ws5,ws6,ws7,ws8,ws9,ws10,ws11,ws12]
route=[]
for ws in wss:
    route.extend(ws)
    for w in ws:
        world.debug.draw_string(w.transform.location, '^', draw_shadow=False,
            color=carla.Color(r=255, g=0, b=255), life_time=600.0,
            persistent_lines=True)
print(len(route))

1297


In [32]:
points=[[-8,140],[100,103],[193,125],[193,200],[165,235],[80,239],[135,210],[40,210],[40,290],[20,190],[95,305],[2,300]]
for i in range(12):
    for j in range(12):
        if i != j:
            #if (i==4 and j==3)or(i==6 and j==3)or(i==7 and j==0)or(i==8 and j==0)or(i==9 and j==0)or(i==11 and j==0):
            #    continue
            print(f"from {i} to {j}:")
            print(len(get_path(carla.Location(x=points[i][0],y=points[i][1],z=0), carla.Location(x=points[j][0],y=points[j][1],z=0))))

from 0 to 1:
418
from 0 to 2:
312
from 0 to 3:
331
from 0 to 4:
320
from 0 to 5:
177
from 0 to 6:
259
from 0 to 7:
116
from 0 to 8:
197
from 0 to 9:
74
from 0 to 10:
263
from 0 to 11:
255
from 1 to 0:
136
from 1 to 2:
447
from 1 to 3:
466
from 1 to 4:
455
from 1 to 5:
312
from 1 to 6:
394
from 1 to 7:
251
from 1 to 8:
332
from 1 to 9:
209
from 1 to 10:
398
from 1 to 11:
390
from 2 to 0:
241
from 2 to 1:
106
from 2 to 3:
571
from 2 to 4:
560
from 2 to 5:
417
from 2 to 6:
499
from 2 to 7:
356
from 2 to 8:
437
from 2 to 9:
314
from 2 to 10:
503
from 2 to 11:
495
from 3 to 0:
318
from 3 to 1:
183
from 3 to 2:
77
from 3 to 4:
467
from 3 to 5:
236
from 3 to 6:
318
from 3 to 7:
175
from 3 to 8:
252
from 3 to 9:
391
from 3 to 10:
318
from 3 to 11:
310
from 4 to 0:
440
from 4 to 1:
305
from 4 to 2:
199
from 4 to 3:
401
from 4 to 5:
247
from 4 to 6:
58
from 4 to 7:
186
from 4 to 8:
173
from 4 to 9:
373
from 4 to 10:
239
from 4 to 11:
231
from 5 to 0:
463
from 5 to 1:
328
from 5 to 2:
222
from 5 

In [53]:
points=[[-8,140],[100,103],[193,125],[193,200],[165,235],[80,239],[135,210],[40,210],[40,290],[20,190],[95,305],[2,300]]
import sys
sys.path.append('/home/youmad55/Downloads/CARLA_0.9.15/PythonAPI/carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner
grp = GlobalRoutePlanner(world.get_map(), 1)
for i in range(12):
    for j in range(12):
        if i != j:
            #if (i==4 and j==3)or(i==6 and j==3)or(i==7 and j==0)or(i==8 and j==0)or(i==9 and j==0)or(i==11 and j==0):
            #    continue
            print(f"from {i} to {j}:")
            print(len(grp.trace_route(carla.Location(x=points[i][0],y=points[i][1],z=0), carla.Location(x=points[j][0],y=points[j][1],z=0))))

from 0 to 1:
513
from 0 to 2:
311
from 0 to 3:
327
from 0 to 4:
342
from 0 to 5:
178
from 0 to 6:
259
from 0 to 7:
117
from 0 to 8:
198
from 0 to 9:
74
from 0 to 10:
263
from 0 to 11:
268
from 1 to 0:
135
from 1 to 2:
446
from 1 to 3:
462
from 1 to 4:
477
from 1 to 5:
313
from 1 to 6:
394
from 1 to 7:
252
from 1 to 8:
333
from 1 to 9:
209
from 1 to 10:
398
from 1 to 11:
403
from 2 to 0:
241
from 2 to 1:
202
from 2 to 3:
568
from 2 to 4:
583
from 2 to 5:
419
from 2 to 6:
500
from 2 to 7:
358
from 2 to 8:
439
from 2 to 9:
315
from 2 to 10:
504
from 2 to 11:
509
from 3 to 0:
317
from 3 to 1:
278
from 3 to 2:
76
from 3 to 4:
490
from 3 to 5:
238
from 3 to 6:
319
from 3 to 7:
177
from 3 to 8:
254
from 3 to 9:
391
from 3 to 10:
319
from 3 to 11:
324
from 4 to 0:
438
from 4 to 1:
399
from 4 to 2:
197
from 4 to 3:
396
from 4 to 5:
247
from 4 to 6:
57
from 4 to 7:
186
from 4 to 8:
173
from 4 to 9:
372
from 4 to 10:
238
from 4 to 11:
243
from 5 to 0:
463
from 5 to 1:
424
from 5 to 2:
222
from 5 

In [29]:
#global planner with u turn

import time
import numpy as np


map=world.get_map()
def key_exists(key, dict_of_dicts):
    for i in dict_of_dicts:
        if key in list(i.keys()):
            return True
    return False

waypoint_tuple_list = map.get_topology()
id_map = dict()
id_map_v = dict()
road_id_to_edge = dict()
graph={}
for waypoint in waypoint_tuple_list:
    x1, y1, x2, y2 = np.round([waypoint[0].transform.location.x, waypoint[0].transform.location.y,waypoint[1].transform.location.x, waypoint[1].transform.location.y], 0)
    road_id, section_id, lane_id = waypoint[0].road_id, waypoint[0].section_id, waypoint[0].lane_id
    if (x1,y1) not in id_map:
        id_map[(x1,y1)] = len(id_map)
        id_map_v[len(id_map_v)]=(x1,y1)
    if (x2,y2) not in id_map:
        id_map[(x2,y2)] = len(id_map)
        id_map_v[len(id_map_v)]=(x2,y2)
    if id_map[(x1,y1)] not in graph :
        graph[id_map[(x1,y1)]]={id_map[(x2,y2)]:waypoint[0]}
    else:
        if id_map[(x2,y2)] not in graph[id_map[(x1,y1)]]:
            graph[id_map[(x1,y1)]].update({id_map[(x2,y2)]:waypoint[0]})
    n1 = id_map[(x1,y1)]
    n2 = id_map[(x2,y2)]
    if road_id not in road_id_to_edge:
        road_id_to_edge[road_id] = dict()
    if section_id not in road_id_to_edge[road_id]:
        road_id_to_edge[road_id][section_id] = dict()
    road_id_to_edge[road_id][section_id][lane_id] = (n1, n2)

def dist(s,e):
    return abs(s[0]-e[0])+abs(s[1]-e[1])
def get_path(start,end):
    wa_start=map.get_waypoint(start)
    road_start_id, section_start_id, lane_start_id = wa_start.road_id, wa_start.section_id, wa_start.lane_id
    (n1_start, n2_start)=road_id_to_edge[road_start_id][section_start_id][lane_start_id] 
    
    wa_end=map.get_waypoint(end)
    road_end_id, section_end_id, lane_end_id = wa_end.road_id, wa_end.section_id, wa_end.lane_id
    (n1_end, n2_end)=road_id_to_edge[road_end_id][section_end_id][lane_end_id]

 #   graph_c=graph.copy()
#    id_map_c=id_map.copy()
#    graph_c['s']={n2_start:wa_start}
    wsl=wa_start.get_left_lane()
    road_start_id, section_start_id, lane_start_id = wsl.road_id, wsl.section_id, wsl.lane_id
    (nu1_start, nu2_start)=road_id_to_edge[road_start_id][section_start_id][lane_start_id]
#    graph_c['su']={nu2_start:wsl}
#    graph_c[n1_end].update({'n':graph_c[n1_end][n2_end]})
    wnl=wa_end.get_left_lane()
    road_start_id, section_start_id, lane_start_id = wnl.road_id, wnl.section_id, wnl.lane_id
    (nu1_end, nu2_end)=road_id_to_edge[road_start_id][section_start_id][lane_start_id]
#    graph_c[nu1_end].update({'nu':graph_c[nu1_end][nu2_end]})
#    graph_c[nu1_end].update({'nu':graph_c[nu1_end][nu2_end]})
    rout=[]
    (x1,y1),(x2,y2) = id_map_v[n2_start],id_map_v[n1_end]
    (x3,y3),(x4,y4) = id_map_v[nu2_start],id_map_v[nu1_end]
    g1 = len(wa_start.next_until_lane_end(1))
    g2 = len(wsl.next_until_lane_end(1))
    h1=dist([x1,y1],[x2,y2])
    if h1>dist([x1,y1],[x4,y4]):
        h1=dist([x1,y1],[x4,y4])

    h2=dist([x3,y3],[x2,y2])
    if h2>dist([x3,y3],[x4,y4]):
        h2=dist([x3,y3],[x4,y4])
        
    f1=g1+h1
    f2=g2+h2
    open = [[f1,g1,h1,['s',n2_start]]]
    open = [[f2,g2,h2,['su',nu2_start]]]
    found = False 
    f_distance={node:float('inf') for node in graph}
    f_distance[n2_start]=f1

    came_from1={node:None for node in graph}
    came_from1[n2_start]=n2_start

    fu_distance={node:float('inf') for node in graph}
    fu_distance[nu2_start]=f2

    came_from_u={node:None for node in graph}
    came_from_u[nu2_start]=nu2_start

    if((nu1_end, nu2_end)==(n1_start, n2_start)) and dist([wa_start.transform.location.x,wa_start.transform.location.y],[wnl.transform.location.x,wnl.transform.location.y])>dist([wa_start.next(1)[0].transform.location.x,wa_start.next(1)[0].transform.location.y],[wnl.transform.location.x,wnl.transform.location.y]):

        rout.append(wa_start)
        while dist([wa_start.transform.location.x,wa_start.transform.location.y],[wnl.transform.location.x,wnl.transform.location.y])>1:
            wa_start=wa_start.next(1)[0]
            rout.append(wa_start)
        rout.append(wa_end)
        return rout

    elif((n1_end, n2_end)==(nu1_start, nu2_start)) and dist([wsl.transform.location.x,wsl.transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y])>dist([wsl.next(1)[0].transform.location.x,wsl.next(1)[0].transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y]):

        rout.append(wa_start)
        while dist([wsl.transform.location.x,wsl.transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y])>1:
            rout.append(wsl)
            wsl=wsl.next(1)[0]
        rout.append(wa_end)
        return rout

    elif((nu1_end, nu2_end)==(nu1_start, nu2_start)) and dist([wsl.transform.location.x,wsl.transform.location.y],[wnl.transform.location.x,wnl.transform.location.y])>dist([wsl.next(1)[0].transform.location.x,wsl.next(1)[0].transform.location.y],[wnl.transform.location.x,wnl.transform.location.y]):

        rout.append(wa_start)
        while dist([wsl.transform.location.x,wsl.transform.location.y],[wnl.transform.location.x,wnl.transform.location.y])>1:
            rout.append(wsl)
            wsl=wsl.next(1)[0]
        rout.append(wa_end)
        return rout

    elif((n1_end, n2_end)!=(n1_start, n2_start)) or dist([wa_start.transform.location.x,wa_start.transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y])<dist([wa_start.next(1)[0].transform.location.x,wa_start.next(1)[0].transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y]):

        while not found:
            if len(open) == 0:
                return "Fail"
                break
            else:
                min_ele = open[0][0]
                index=0
                for i in range(1, len(open)):
                    if open[i][0] < min_ele:
                        min_ele = open[i][0]
                        index=i
                next=open.pop(index)
                path = next[3].copy()
                #print(path[-1])
                g = next[1]
                for i in graph[path[-1]]:
                    #print(i)
                    (xf,yf)=id_map_v[i]
                    g2=len(graph[path[-1]][i].next_until_lane_end(1))+g
                    #print(g2)
                    h=dist([xf,yf],[x2,y2])
                    if h>dist([xf,yf],[x4,y4]):
                        h=dist([xf,yf],[x4,y4])
                    f2=g2+h
                    if path[0]=='s':
                        came_from=came_from1
                    elif path[0]=='su':
                        came_from=came_from_u
                    if came_from[i]==None:
                        f_distance[id_map[(xf,yf)]]=f2
                        came_from[id_map[(xf,yf)]]=path[-1]
                        fpath=path.copy()
                        fpath.append(i)
                        open.append([f2,g2,h,fpath])
                        
                        if xf == x2 and yf == y2:
                            found = True
                            #came_from['n']=id_map[(x2,y2)]
                            #return came_from
                            wa=map.get_waypoint(carla.Location(x=x2,y=y2,z=0))
                            rout.append(wa)
                            while dist([wa.transform.location.x,wa.transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y])>1:
                                wa=wa.next(1)[0]
                                rout.append(wa)

                            if path[0]=='su':
                                n2_start=nu2_start
                            while n1_end != n2_start:
                                n=came_from[n1_end]
                                ws=graph[n][n1_end].next_until_lane_end(1)
                                ws.extend(rout)
                                rout=ws.copy()
                                n1_end=n
                            if path[0]=='su':
                                ws=[wa_start]
                                ws.extend(wsl.next_until_lane_end(1))
                            elif path[0]=='s':
                                ws=wa_start.next_until_lane_end(1)
                            ws.extend(rout)
                            rout=ws.copy()
                            break

                        elif xf == x4 and yf == y4:
                            found = True
                            #came_from['n']=id_map[(x2,y2)]
                            #return came_from
                            wa=map.get_waypoint(carla.Location(x=x4,y=y4,z=0))
                            rout.append(wa)
                            while dist([wa.transform.location.x,wa.transform.location.y],[wnl.transform.location.x,wnl.transform.location.y])>1:
                                wa=wa.next(1)[0]
                                rout.append(wa)

                            rout.append(wa_end)
                            if path[0]=='su':
                                n2_start=nu2_start
                            while nu1_end != n2_start:
                                n=came_from[nu1_end]
                                ws=graph[n][nu1_end].next_until_lane_end(1)
                                ws.extend(rout)
                                rout=ws.copy()
                                nu1_end=n
                            if path[0]=='su':
                                ws=[wa_start]
                                ws.extend(wsl.next_until_lane_end(1))
                            elif path[0]=='s':
                                ws=wa_start.next_until_lane_end(1)
                            ws.extend(rout)
                            rout=ws.copy()
                            break
    else:
        rout.append(wa_start)
        while dist([wa_start.transform.location.x,wa_start.transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y])>1:
            wa_start=wa_start.next(1)[0]
            rout.append(wa_start)
        rout.append(wa_end)
    return rout


points=[[-8,140],[100,103],[193,125],[193,200],[165,235],[80,239],[135,210],[40,210],[40,290],[20,190],[95,305],[2,300]]

start=carla.Location(x=points[0][0],y=points[0][1],z=0)
world.debug.draw_string(start, 's', draw_shadow=False,
            color=carla.Color(r=0, g=0, b=255), life_time=60.0,
            persistent_lines=True)
end=carla.Location(x=points[1][0],y=points[1][1],z=0)
world.debug.draw_string(end, 'e', draw_shadow=False,
            color=carla.Color(r=0, g=0, b=255), life_time=60.0,
            persistent_lines=True)
route=get_path(start,end)
#print(route)
for i in route:
    world.debug.draw_string(i.transform.location, '*', draw_shadow=False,
            color=carla.Color(r=0, g=255, b=255), life_time=60.0,
            persistent_lines=True)
    #time.sleep(1)
#end=map.get_waypoint(end)
#road_end_id, section_end_id, lane_end_id = end.road_id, end.section_id, end.lane_id
#(n1, n2)=road_id_to_edge[road_end_id][section_end_id][lane_end_id]

#print(n1)
